In [1]:
from pyspark.sql import Row, SparkSession

In [2]:
spark = SparkSession.builder.appName("test").master("local[*]").getOrCreate()

In [3]:
df = spark.read.format("csv").option("header", True)\
.option("inferschema", True).load("../DataDummy/data/DataBricks_Training_double.csv")

In [5]:
#select total price for each course and Venue
from pyspark.sql.functions import sum

fee_venue_course = df.groupBy("Name", "Venue").agg(sum("Fee").alias("Total_Fee"))\
.select("Venue", "Name", "Total_Fee")

In [10]:
# select the total prices for each venue
from pyspark.sql.functions import lit
fee_for_venue = df.groupBy("Venue").agg(sum("Fee").alias("Total_Fee"))\
.select("Venue", lit("Total price from this value").alias("Name"), "Total_Fee")

fee_for_venue.show(2)

+---------+--------------------+---------+
|    Venue|                Name|Total_Fee|
+---------+--------------------+---------+
|Singapore|Total price from ...|    92000|
|Frankfurt|Total price from ...|    74000|
+---------+--------------------+---------+
only showing top 2 rows



In [12]:
# select the total price for each course
courseTotal = df.groupBy("Name").agg(sum("Fee").alias("Total_Fee"))\
.select(lit("Total price from this venue").alias("Venue"), "Name", "Total_Fee")

courseTotal.show(2)

+--------------------+--------------+---------+
|               Venue|          Name|Total_Fee|
+--------------------+--------------+---------+
|Total price from ...|     HE Hadoop|    18000|
|Total price from ...|HE SAS BigData|    24000|
+--------------------+--------------+---------+
only showing top 2 rows



In [19]:
#show all the prices together using union
from pyspark.sql.functions import asc_nulls_last, asc_nulls_first

fee_venue_course.union(fee_for_venue).union(courseTotal)\
.sort(asc_nulls_first("Venue"), asc_nulls_first("Name"))\
.show()

+-------+--------------------+---------+
|  Venue|                Name|Total_Fee|
+-------+--------------------+---------+
|Beijing|HE Datascience Al...|    24000|
|Beijing|  HE EMC Datascience|    24000|
|Beijing|HE Google Cloud A...|    14000|
|Beijing|          HE GraphDB|    24000|
|Beijing|        HE Hybernate|    14000|
|Beijing|Total price from ...|   100000|
| Berlin|      HE Hadoop Hive|    12000|
| Berlin|              HE Hue|    12000|
| Berlin|       HE Oracle DBA|    20000|
| Berlin|           HE VMWare|    14000|
| Berlin|Total price from ...|    58000|
|Chennai|   HE AWS Networking|    20000|
|Chennai|           HE CCA159|    16000|
|Chennai|        HE Mac Admin|    20000|
|Chennai|     HE Redhat Admin|    20000|
|Chennai|      HE Spark Graph|    12000|
|Chennai|Total price from ...|    88000|
|  Dubai|  HE AWS Elasticache|    24000|
|  Dubai|HE Cloudera Hadoo...|    20000|
|  Dubai|           HE Jquery|    18000|
+-------+--------------------+---------+
only showing top